# HV19.09 Santas Quick Response 3.0

## Introduction

We get the following picture of a railway station:

![Railway Station](train.jpg)

And a somewhat broken QR code:

![QR Code](qr.png)

## Initial research

A Google reverse image search reveals a [Wikipedia article](https://en.wikipedia.org/wiki/Rule_30) about "Rule 30", a cellular automaton similar to e.g. Conway's Game of Life:

![Rule 30](rule30.png)

Pixels are set based on the pattern above them:

| 111 | 110 | 101 | 100 | 011 | 010 | 001 | 000 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 0   | 0   | 0   | 1   | 1   | 1   | 1   | 0   |

## Loading QR code

First, let's try to load the QR code into Python via [Pillow](https://pillow.readthedocs.io/).
The QR code is 33x33 squares big, one square is 5x5px.

In [3]:
from PIL import Image

CNT = 33  # squares in the QR-code

def read():
    qr = Image.open("qr.png")
    data = [[] for _ in range(CNT)]
    for sq_y in range(CNT):
        for sq_x in range(CNT):
            pixel = qr.getpixel((sq_x * 5, sq_y * 5))
            val = {0: 1, 255: 0}[pixel]
            data[sq_y].append(val)
        
    return data

def display(data):
    chars = {1: '*', 0: ' '}
    for line in data:
        print(''.join(chars[c] for c in line))

In [4]:
data = read()
display(data)

*******   *  * **   ***   *******
*     * **  ** *****  * * *     *
* *** * ** * *** *****  * * *** *
* *** * **   * * * ****   * *** *
* *** * **** *     **  *  * *** *
*     * ****  ****  ** *  *     *
******* * **  *** *   *** *******
         ***** ** *  **          
  *  ****   * *   *     *  ***** 
  **** ** * *** * *   **       **
* * * **   * * * * ** * ******* *
 *    **  * *     * * ***  ***   
    **  *   *  *    **  *** ** **
 ***    ** * * * *     ** * ***  
   ** *   *  **     *** ***   ***
 **   * *  *    * * **** ****** *
  *   *    *  **  *      * *  ** 
****  * **  * *  * *   * **   * *
* *  ****** **  ** *  *  **  ****
*  *   **  * *     *** * ***  ***
 * **   * ***** ***  *  *******  
*** *   * **  **  ****  *  ***** 
*** *    ** **** * ***  *  * * * 
* *  ****  ***** * * * * * ******
*  ***  ** *  **  *  *** *    * *
**  *** * ** * *   *   *  * *  * 
 *   *****    *** ** * **  ***   
  *  * **** *** *    *   **  **  
*    **   *   *** *   * **** ****
 * **   ** ** 

## First idea

Looking at the QR code, the data in the top corners seems to be valid (so we probably don't need to touch it), while the data in the bottom corners clearly isn't a valid QR code. When taking a closer look, we also see some other patterns that are missing - from Wikipedia:

![QR code structure](qr-structure.svg)

Thus, we maybe need to use the rule30 data to toggle bits, so that we leave the top corners alone but modify the bottom corners?

I'll use the [rule30](https://pypi.org/project/rule30/) Python library to generate Rule30 patterns.

In [5]:
import rule30

r30 = rule30.Automaton(rows=CNT)
print(r30.string())

00000000000000000000000000000000100000000000000000000000000000000
00000000000000000000000000000001110000000000000000000000000000000
00000000000000000000000000000011001000000000000000000000000000000
00000000000000000000000000000110111100000000000000000000000000000
00000000000000000000000000001100100010000000000000000000000000000
00000000000000000000000000011011110111000000000000000000000000000
00000000000000000000000000110010000100100000000000000000000000000
00000000000000000000000001101111001111110000000000000000000000000
00000000000000000000000011001000111000001000000000000000000000000
00000000000000000000000110111101100100011100000000000000000000000
00000000000000000000001100100001011110110010000000000000000000000
00000000000000000000011011110011010000101111000000000000000000000
00000000000000000000110010001110011001101000100000000000000000000
00000000000000000001101111011001110111001101110000000000000000000
00000000000000000011001000010111000100111001001000000000000000000
0000000000

This looks like something we can use to try that theory, but it's not a square... Given that the QR code is 33x33 fields big, you'd think you can just start with the middle pixel in the first row. This doesn't result in a working QR code however.

After my original (failed) approach, I fired up GIMP and tried to create a "diff" of what you'd need to change in the QR code to get the fixed patterns. Dark read means "is 0 but should be 1", light red means "is 1 but should be 0".

![QR code diff](qr-diff.png)

I couldn't find those patterns in rule30, so I might have done a mistake at some point. However, it later turned out that the start is shifted by one pixel... A challenge hint "centering is hard" was released later, but I'm assuming this wasn't originally intended.

In [6]:
len(r30.matrix[0]), len(r30.matrix)  # width, height

(65, 33)

In [7]:
def flip_bits(data):
    diff = 65 - CNT
    # For some reason, it's not centered in the challenge...
    left = diff // 2 - 1
    right = diff // 2 + 1
    for y in range(CNT):
        r30_line = r30.matrix[y][left:-right]
        # print(r30_line)
        for x in range(CNT):
            data[y][x] ^= r30_line[x]

In [8]:
data = read()
flip_bits(data)

This results in something which looks like a QR-code!

In [9]:
display(data)

*******   *  * ***  ***   *******
*     * **  ** *   *  * * *     *
* *** * ** * ** *** **  * * *** *
* *** * **   **   *  **   * *** *
* *** * ****  *  * *** *  * *** *
*     * *******   *   **  *     *
******* * * * * * * * * * *******
         *  * *   ***  **        
  *  ****** ***  * *    ** ***** 
  **** * ***     ** * *****    **
* * * * *    * ****  *** ** *** *
 *       * *   **   * * ***  *   
    * * **  ***   ******* * *  **
 ***** *  ***  ** * ******       
      **  * ** **    ***  * * * *
 * * * *    * * ****     *     * 
 *   **  **     ****    * **  ** 
  * **       *   *  *   ****  *  
  ** *** * * * ****  * *   ****  
*** *     **  *******  *  ** *  *
   ******     * ***** *    *  *  
     *   * *   *    **     * * **
***   *******     ** * * *  *****
  **** **** * **   ** *  *  ** * 
*** ****   * * *** ***********   
        *  **  **  * *  *   *  **
******* * * *    ****   * * ** **
*     * * *  *  * **** **   ** * 
* *** *  * **   **   * ******* **
* *** *    ** 

In [13]:
import itertools

def write_img(data):
    img = Image.new(mode='1', size=(CNT, CNT))
    for y, line in enumerate(data):
        for x, val in enumerate(line):
            img.putpixel((x, y), not val)
    
    return img.resize((CNT * 5, CNT * 5))

img = write_img(data)
img.save("result.png")

This gives us the QR code. For some extra h4xx0r points, let's use [zbarlight](https://github.com/Polyconseil/zbarlight/) to get the flag as well.

![Resulting QR-Code](result.png)

In [14]:
import zbarlight
    
codes = zbarlight.scan_codes(['qrcode'], img)
print(codes[0].decode('ascii'))

HV19{Cha0tic_yet-0rdered}
